Creating the Final_dataset.csv file for Open Data Mashup

The purpose of this jupyter notebook is to create my final dataset file for my Open Data Mashups project. This file is titled 'Final_dataset.csv.' Preparation and cleaning of intermediate datasets was required before completing this final dataset. Below is a list of all input files used and read into this juypter notebook: 

- Intermediate_PR_dataset.csv
- Unemployment_ChiHealthAtlas.csv
- Violent_crime_in_public_spaces_ChiHealthAtlas.csv
- Medianincome_ChiHealthAtlas.csv
- altnames_csv.csv


In this notebook we will be creating a dictionary that will be used to store data about Chicago Community Areas. This dictionary will then be used to loop through the press release data compiled as an intermediate dataset. Finally, this script will write out a CSV file as the final dataset used in for Tableau visualization. 

To begin, first we load our the libraries and packages we will need and initalize a dictionary object. Our dictionary will be called "n_data".

In [2]:
import csv
import json

n_data = {} # This creates a dictionary object

Next, we open all the files required for this script. These are the intermediate datasets listed above. We will read in 5 intermediate datasets and save each in an object as a list of lists.

In [3]:
f = open('Intermediate_PR_dataset.csv')
pr_text = csv.reader(f, delimiter=',', )
next(pr_text) # move past headers
next(pr_text) #move past headers
pr_list = [r for r in pr_text] #list of lists

f2 = open('Unemployment_ChiHealthAtlas.csv')
unemployment_data = csv.reader(f2, delimiter=',', )
unemployment_list = [r for r in unemployment_data]


f3 = open('Violent_crime_in_public_spaces_ChiHealthAtlas.csv')
crime_data = csv.reader(f3, delimiter=',', )
crime_list = [r for r in crime_data]


f4 = open('Medianincome_ChiHealthAtlas.csv')
median_income_data = csv.reader(f4, delimiter=',', )
median_income_list = [r for r in median_income_data]


with open('altnames_csv.csv', 'r') as infile:
    csvin = csv.reader(infile)
    data = [r for r in csvin] 

Next, we assign each Chicago Community Area to be a keys of the dictionary by looping through the 'altnames_csv.csv' file. We also create empty values in the dictionary for the values 'appears' and 'pr_id.'

In [4]:
for nhood in data:
    std_name = nhood[0] #For each list in my list of lists of altnames, grab the first value
    n_data[std_name] = {"names" : nhood, "appears": 0, "pr_id": []} #Set the standard neighborhood name as the key and the alt names as the value of names, fills other values with empty data


Next, we loop through the column in the intermediate dataset 'Intermediate_PR_dataset.csv' that contains the body text of each press release. This script counts the number of appearances of Chicago Community Names within those press releases and saves that count to the value 'appears' as well as the unique identifier for each press release under the value 'pr_id'.

In [5]:
for pr in pr_list:
    pr_id = pr[0]
    title = pr[1]
    text = pr[2]
    date = pr[3]
    for nid in n_data.keys(): #Looks up all the std names
        allnames = n_data[nid]["names"] #Gives a list of the names
        for name in allnames:
            if name in text:
                n_data[nid]["pr_id"].append(pr_id)
                n_data[nid]["appears"] += 1
                break

Once the above data is stored in the dictionary, what is left to do is store the rest of the data collected about Chicago Community Areas that I have collected previously. This includes all of the Chicago Health Atlas health indicator data like median income, unemployment rate, and crimes committed in public for each Community Area. In the following script I add each of these health indicators to their respective Community Areas by saving each column to an object and inserting them into the value that they belong to in the dictionary. 

In [6]:


for column in unemployment_list:
    std_name = column[0]
    geo_id = column[1]
    percent = column[2]
    if "-" in std_name:
        std_name = std_name.split("-")[-1]
    n_data[std_name]["geo_id"]= geo_id
    n_data[std_name]["percent"]= percent


for item in crime_list:
    year = item[0]
    std_name = item[1]
    crime = item[3]
    if "-" in std_name:
        std_name = std_name.split("-")[-1]
    n_data[std_name]["crime"]= crime

for item4 in median_income_list:
    std_name = item4[0]
    geo_id = item4[1]
    median_income = item4[2]
    if "-" in std_name:
        std_name = std_name.split("-")[-1]
    n_data[std_name]["median income"]= median_income
    n_data[std_name]["geo_id"]= geo_id

In this final cell, I am writing out my dictionary to different data formats. First, I write it out as a JSON file titled 'nhooddata.json' and secondly I write it out as a CSV titled 'Final_dataset.csv.' 

In [7]:
for nid, vals in n_data.items():
    print(nid, vals)
    # print(nid, vals['appears'], vals['pr_id'], vals['unemployment'])

with open('nhooddata.json', 'w') as fout:
    json.dump(n_data, fout, indent = 4)

#Aileen attempting to write this JSON into a CSV
#n_data_parsed = json.loads()


outputfile = open('Final_dataset.csv', 'w')
csvwriter = csv.writer(outputfile)

columnTitleRow = "std_name, geo_id, percent_unemployment, crime, median_income, appears, pr_id \n"
outputfile.write(columnTitleRow)

for key in n_data.keys():
    std_name = key
    geo_id = str(n_data[key]["geo_id"])
    percent = str(n_data[key]["percent"])
    crime = str(n_data[key]["crime"])
    median_income = str(n_data[key]["median income"])
    appears = str(n_data[key]["appears"])
    pr_id = '"' + str(n_data[key]["pr_id"]) + '"'
    row = std_name + "," + geo_id + "," + percent + "," + crime + "," + median_income + "," + appears + "," + pr_id + "," + "\n"
    outputfile.write(row)

outputfile.close()




Rogers Park {'names': ['Rogers Park', 'East Rogers Park', 'Loyola'], 'appears': 18, 'pr_id': ['39', '160', '218', '275', '296', '313', '345', '378', '416', '541', '662', '720', '777', '798', '815', '847', '880', '918'], 'geo_id': '1', 'percent': '9.1', 'crime': '120', 'median income': '36114'}
West Ridge {'names': ['West Ridge', 'West Rogers Park', 'Peterson Park', 'Arcadia Terrace'], 'appears': 4, 'pr_id': ['309', '313', '811', '815'], 'geo_id': '2', 'percent': '9.2', 'crime': '139', 'median income': '53301'}
Uptown {'names': ['Uptown', 'Buena Park', 'Argyle Street', 'Margate Park', 'Sheridan Park'], 'appears': 8, 'pr_id': ['9', '105', '275', '334', '511', '607', '777', '836'], 'geo_id': '3', 'percent': '8', 'crime': '155', 'median income': '47651'}
Lincoln Square {'names': ['Lincoln Square', 'Ravenswood', 'Ravenswood Gardens', 'Rockwell Crossing'], 'appears': 4, 'pr_id': ['221', '338', '723', '840'], 'geo_id': '4', 'percent': '6', 'crime': '69', 'median income': '70525'}
North Center